# Тестирование эмбедингов

# Rusvectors

In [ ]:
import pickle
import zipfile
import wget
from preproccess_text import tag_ud

In [ ]:
сorpus_file = "./dataset/corpus.txt"
# with open(corpus_file, "wb") as f:
#     pickle.dump(corpus, f)

with open(сorpus_file, "rb") as f:   # Unpickling
    corpus = pickle.load(f)

In [ ]:
corpus = []
stops = set(stopwords.words('russian'))
root_path = 'C:\\Users\\edbon\\devproj\\faiky-tails\\dataset\\raw\\'
modelfile = './udpipe_syntagrus.model'
with open(root_path + '001 Арысь - поле.txt', 'r', encoding='utf-8') as f:
    doc = tag_ud(text=f.read(), modelfile=modelfile, stop_words=stops, keep_pos=True, keep_punct=False)
corpus.append(doc)

In [ ]:
model_path = 'c:/models/rusvector/'
model_file = model_path + '180.zip'
if not os.path.exists(model_file):
    model_url = 'http://vectors.nlpl.eu/repository/11/180.zip'
    m = wget.download(model_url, out=model_path)
    model_file = model_path + model_url.split('/')[-1]

with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model_rv = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [ ]:
print(stops)

{'почти', 'над', 'тогда', 'с', 'при', 'вдруг', 'теперь', 'без', 'после', 'про', 'сейчас', 'как', 'по', 'кто', 'есть', 'тем', 'вот', 'что', 'мой', 'какая', 'она', 'тут', 'ведь', 'нет', 'сам', 'всегда', 'всю', 'впрочем', 'перед', 'потому', 'зачем', 'свою', 'и', 'а', 'так', 'чтобы', 'не', 'да', 'была', 'тебя', 'ему', 'вам', 'может', 'конечно', 'вы', 'со', 'из', 'то', 'него', 'чем', 'иногда', 'более', 'мне', 'этого', 'на', 'один', 'ее', 'разве', 'того', 'два', 'эту', 'об', 'нее', 'всех', 'там', 'под', 'будет', 'были', 'его', 'все', 'ли', 'ничего', 'они', 'уж', 'чтоб', 'опять', 'меня', 'эти', 'всего', 'куда', 'моя', 'лучше', 'много', 'у', 'вас', 'для', 'мы', 'или', 'ней', 'во', 'чего', 'надо', 'какой', 'уже', 'чуть', 'от', 'хорошо', 'еще', 'совсем', 'был', 'никогда', 'тот', 'он', 'можно', 'ни', 'этой', 'ну', 'нас', 'о', 'этом', 'до', 'себя', 'через', 'раз', 'быть', 'больше', 'ж', 'за', 'бы', 'но', 'потом', 'ты', 'хоть', 'между', 'этот', 'даже', 'когда', 'будто', 'такой', 'нибудь', 'том', 'н

In [ ]:
# надо суффикс проставлять для слова с частеречием

bad_words = set()
for word in corpus[0]:
    try:
        model_rv[word]
    except KeyError as err:
        bad_words.add(word)
print("не в словаре:", bad_words)

for word in bad_words:
    w = word.split('_')[0]
    
    if w in stops:
        print("стоп слова:", word, end=' ')

{'арысь_NOUN', 'весь_DET', 'невзлюбить_PROPN', 'стариковый_ADJ', 'свой_DET', 'падчерицыный_ADV', 'вместо_ADP', 'никто_PRON'}
арысь
весь
невзлюбить
стариковый
свой
падчерицыный
вместо
никто


## Результаты:

* в готовом эмбединге отсутствуют часть слов из корпуса, надо смотреть возможности дообучения

# Исследование Word2Vec

In [8]:
import numpy as np
import timeit

In [33]:
f = lambda x: np.exp(x)
vf = np.vectorize(f)

a = np.array([3.0, 1.0, -3.0])
softmax = np.exp(a) / np.exp(a).sum()
print(type(softmax), softmax)

n = 10000
t1 = timeit.timeit('np.exp(a) / np.exp(a).sum()', 'from __main__ import a, divider, np', number=n)
t2 = timeit.timeit('vf(a) / np.exp(a).sum()', 'from __main__ import a, divider, np, vf', number=n)
t3 = timeit.timeit('f(a) / np.exp(a).sum()', 'from __main__ import a, divider, np, f', number=n)


print("direct np func=", round(t1, 3))
print("np vectorize=", round(t2, 3))
print("direct my f on np func=", round(t3, 3))

<class 'numpy.ndarray'> [0.87887824 0.11894324 0.00217852]
direct np func= 0.031
np vectorize= 0.128
direct my f on np func= 0.029
